In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))  # Add project root to Python path
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from src.utils import load_train_data, load_test_data, resumetable, col_value_counts
train_df = load_train_data()
test_df = load_test_data()

In [4]:
all_data = pd.concat([train_df, test_df], ignore_index=True)
all_data = all_data.drop('target', axis=1)

In [5]:
all_features = all_data.columns
all_features

Index(['id', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtyp

In [6]:
all_data

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,2,2,5,1,0,0,1,0,0,...,9,1,5,8,0,1,1,0,0,1
1,9,1,1,7,0,0,0,0,1,0,...,3,1,1,9,0,1,1,0,1,0
2,13,5,4,9,1,0,0,0,1,0,...,4,2,7,7,0,1,1,0,1,0
3,16,0,1,2,0,0,1,0,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,2,0,1,0,1,0,0,0,...,3,1,1,3,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488023,1488022,0,1,6,0,0,0,1,0,0,...,4,2,3,4,0,1,0,0,1,0
1488024,1488023,5,3,5,1,0,0,0,1,0,...,6,2,2,11,0,0,1,1,0,0
1488025,1488024,0,1,5,0,0,1,0,0,0,...,5,2,2,11,0,1,1,0,0,0
1488026,1488025,6,1,5,1,0,0,0,0,1,...,1,1,2,7,1,1,0,0,0,0


In [7]:
all_data = all_data.drop(columns=['ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_13_bin', 'ps_ind_18_bin', 'ps_ind_14'])

In [8]:
for feature in all_features:
    if ('calc' in feature):
        all_data = all_data.drop(columns=[feature])

In [9]:
all_data.columns

Index(['id', 'ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_12_bin', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03', 'ps_car_01_cat',
       'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat', 'ps_car_05_cat',
       'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat', 'ps_car_09_cat',
       'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11', 'ps_car_12', 'ps_car_13',
       'ps_car_14', 'ps_car_15'],
      dtype='object')

In [10]:
all_data_df = resumetable(all_data)

Data shape: (1488028, 33)


In [11]:
missing_data = all_data_df[all_data_df['Missing'] > 0]
missing_data.sort_values(by='Missing', ascending=False)

,Data Type,Missing,Nunique,Feature Type
ps_car_03_cat,int64,1028142,3,Nominal
ps_car_05_cat,int64,666910,3,Nominal
ps_reg_03,float64,269456,5077,Continuous
ps_car_14,float64,106425,905,Continuous
ps_car_07_cat,int64,28820,3,Nominal
ps_ind_05_cat,int64,14519,8,Nominal
ps_car_09_cat,int64,1446,6,Nominal
ps_ind_02_cat,int64,523,5,Nominal
ps_car_01_cat,int64,267,13,Nominal
ps_ind_04_cat,int64,228,3,Nominal


In [12]:
col_value_counts(all_data, 'ps_car_11')

--- ps_car_11 value_counts ---
ps_car_11
 3    797388
 2    473339
 1    152661
 0     64634
-1         6
Name: count, dtype: int64


In [13]:
# 중앙값 대체 
continuois_features_to_impute = ['ps_reg_03', 'ps_car_14', 'ps_car_12']
for feature in continuois_features_to_impute:
    all_data[feature] = all_data[feature].replace(-1, np.nan)
    median_val = all_data[feature].median()
    all_data[feature] = all_data[feature].fillna(median_val)

In [14]:
all_data_df = resumetable(all_data)
missing_data = all_data_df[all_data_df['Missing'] > 0]
missing_data.sort_values(by='Missing', ascending=False)

Data shape: (1488028, 33)


,Data Type,Missing,Nunique,Feature Type
ps_car_03_cat,int64,1028142,3,Nominal
ps_car_05_cat,int64,666910,3,Nominal
ps_car_07_cat,int64,28820,3,Nominal
ps_ind_05_cat,int64,14519,8,Nominal
ps_car_09_cat,int64,1446,6,Nominal
ps_ind_02_cat,int64,523,5,Nominal
ps_car_01_cat,int64,267,13,Nominal
ps_ind_04_cat,int64,228,3,Nominal
ps_car_02_cat,int64,10,3,Nominal
ps_car_11,int64,6,5,Ordinal


In [15]:
all_data['ps_car_11'] = all_data['ps_car_11'].replace(-1, 3)

In [16]:
all_data

,id,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
0,7,2,2,5,1,0,0,1,0,0,...,1,0,0,1,12,2,0.400000,0.883679,0.370810,3.605551
1,9,1,1,7,0,0,0,0,1,0,...,1,1,2,1,19,3,0.316228,0.618817,0.388716,2.449490
2,13,5,4,9,1,0,0,0,1,0,...,1,1,2,1,60,1,0.316228,0.641586,0.347275,3.316625
3,16,0,1,2,0,0,1,0,0,0,...,1,1,3,1,104,1,0.374166,0.542949,0.294958,2.000000
4,17,0,2,0,1,0,1,0,0,0,...,1,1,2,1,82,3,0.316070,0.565832,0.365103,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1488023,1488022,0,1,6,0,0,0,1,0,0,...,0,1,0,1,18,2,0.565685,1.528774,0.565685,3.605551
1488024,1488023,5,3,5,1,0,0,0,1,0,...,1,0,0,1,83,2,0.424264,1.040567,0.418569,3.741657
1488025,1488024,0,1,5,0,0,1,0,0,0,...,1,0,1,1,17,2,0.374166,0.968992,0.390384,3.741657
1488026,1488025,6,1,5,1,0,0,0,0,1,...,1,1,0,1,28,3,0.387298,0.725125,0.378153,2.645751


In [17]:
num_train_df = len(train_df)
X = all_data[:num_train_df]
X_test = all_data[num_train_df:]
y = train_df['target'].values

In [18]:
def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    n_samples = y_true.shape[0]
    L_mid = np.linspace(1 / n_samples, 1, n_samples)

    # 예측값에 대한 지니계수 
    pred_order = y_true[y_pred.argsort()]
    L_pred = np.cumsum(pred_order) / np.sum(pred_order)
    G_pred = np.sum(L_mid - L_pred)

    # 예측이 완벽할 때 지니계수 
    true_order = y_true[y_true.argsort()]
    L_true = np.cumsum(true_order) / np.sum(true_order)
    G_true = np.sum(L_mid - L_true)

    return G_pred / G_true

In [19]:
# 기본적으로 lightgbm에서 내장 평가지표 지원하지만, 경진대회 요구 평가지표 코드
def gini(preds, dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels, preds), True

In [20]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=44)

In [21]:
oof_val_preds = np.zeros(X.shape[0]) # OOF에서 훈련된 모델로 검증 데이터 타깃값을 예측한 확률 담을 배열
oof_test_preds = np.zeros(X_test.shape[0]) # OOF에서 훈련된 모델로 테스트 데이터 타깃값을 예측한 확률 담을 배열
'''
	1.	전체 데이터 X.shape[0]은 100개
	2.	oof_val_preds = np.zeros(100) ← 먼저 100개의 빈 공간 마련
	3.	각 Fold 순회 시 마다:
    	• Fold 1: train 80개, val 20개
    	• 모델은 train으로 학습, val로 예측 → 예측 결과 20개 나옴
    	• 이 20개의 예측값을 원래 val_index 위치에 넣어줌
'''

'\n\t1.\t전체 데이터 X.shape[0]은 100개\n\t2.\toof_val_preds = np.zeros(100) ← 먼저 100개의 빈 공간 마련\n\t3.\t각 Fold 순회 시 마다:\n    \t• Fold 1: train 80개, val 20개\n    \t• 모델은 train으로 학습, val로 예측 → 예측 결과 20개 나옴\n    \t• 이 20개의 예측값을 원래 val_index 위치에 넣어줌\n'

In [22]:
params = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'random_state': 44,
}
# params 딕셔너리에는 모델의 핵심 하이퍼 파라미터 사용
# 함수의 직접적인 인자로는 학습 실행과 관련된 제어요소 사용

import lightgbm as lgb

for idx, (train_idx, valid_idx) in enumerate(folds.split(X,y)):
    print(f'Fold {idx+1} / Fold {folds.n_splits}')
# split(X,y): folds 객체의 메서드 
# x와 y 데이터를 K개의 폴드로 분할 -> 실제 분할은 x, 데이터의 인덱스(위치 정보) 생성하여 반환
# enumerate(): 이터러블(iterable) 객체(folds.split(X,y)가 반환하는 이터레이터)를 순회하면서, 각 항목에 인덱스(순서 번호)를 부여
# for idx, (train_idx, valid_idx) in ...: (train_idx, valid_idx): folds.split(X,y)가 반환하는 튜플의 언패킹(unpacking)
# train_idx: 훈련 데이터 샘플들의 인덱스 배열, valid_idx: 검증 데이터 샘플들의 인덱스 배열
# idx: enumerate 함수가 부여하는 현재 반복의 인덱스 번호(0 ~ n_splits-1)
    
    # 훈련용/검증용 데이터 설정 
    X_train, y_train = X.iloc[train_idx], y[train_idx]
    X_valid, y_valid = X.iloc[valid_idx], y[valid_idx]

    # LightGBM 전용 데이터셋 설정 
    dtrain = lgb.Dataset(X_train, y_train)
    dvalid = lgb.Dataset(X_valid, y_valid)

    # LightGBM 모델 훈련
    lgb_model = lgb.train(
        params = params,
        train_set = dtrain,
        valid_sets = [dvalid],
        feval = gini,
        num_boost_round = 1000,
        callbacks=[
            lgb.early_stopping(stopping_rounds=100, verbose=False), 
            # early_stopping_rounds 대체
            lgb.log_evaluation(period=100) 
            # 학습 진행 상황 출력
            # Boosting Round(트리의 개수)가 100씩 증가하며 
            # 특정 번째 트리를 학습한 시점에서 검증 성능 출력
        ]
    )
    # 이 코드 마무리 되었을 때 fold 1에 대한 학습과 종료 

    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    # 이 코드 실행되었을 때 fold 1에 대한 예측치 모두 리스트에 채워짐
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'Fold {idx+1} gini score: {gini_score}\n')

    # 출력값 binary_logloss: params에 입력된 binary 인식하여 모델의 예측 확률과 실제 레이블의 일치도 측정, 낮을수록 정확


Fold 1 / Fold 5
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008357 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1422
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 33
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
[100]	valid_0's binary_logloss: 0.15333	valid_0's gini: 0.267137
[200]	valid_0's binary_logloss: 0.152399	valid_0's gini: 0.27988
[300]	valid_0's binary_logloss: 0.15203	valid_0's gini: 0.28443
[400]	valid_0's binary_logloss: 0.151837	valid_0's gini: 0.287787
[500]	valid_0's binary_logloss: 0.151714	valid_0's gini: 0.290317
[600]	valid_0's binary_logloss: 0.151683	valid_0's gini: 0.290662
[700]	valid_0's binary_loglos

In [23]:
print('OOF Data Gini:', eval_gini(y, oof_val_preds))

OOF Data Gini: 0.28223439180310655


In [24]:
submission = pd.DataFrame({'id': test_df['id'], 'target': oof_test_preds})
submission['target'] = oof_test_preds
submission.to_csv('submission.csv', index=False)